# Generate flux table for pcigale

Test Fors2 IO for `fors2pcigale` package

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2024-02-04
- last update : 2024-02-04

  Use ``pcigale`` python environnement

## File to generate for cigale input

In [24]:
# File containing the input data. The columns are 'id' (name of the
# object), 'redshift' (if 0 the distance is assumed to be 10 pc),
# 'distance' (Mpc, optional, if present it will be used in lieu of the
# distance computed from the redshift), the filter names for the fluxes,
# and the filter names with the '_err' suffix for the uncertainties. The
# fluxes and the uncertainties must be in mJy for broadband data and in
# W/m² for emission lines. Fluxes can be positive or negative. Upper
# limits are indicated with a negative value for the uncertainty. In
# case some fluxes are missing for some entries, they can be replaced
# with NaN. This file is optional to generate the configuration file, in
# particular for the savefluxes module.

### name of filters in CIGALE
    redshift distance 
    galex.FUV galex.FUV_err galex.NUV galex.NUV_err 
    sdss.up sdss.up_err sdss.gp sdss.gp_err  sdss.rp sdss.rp_err sdss.ip sdss.ip_err 
   
    vista.vircam.Z                                                                                             
    vista.vircam.Y                                                                                         
    vista.vircam.J                                                                                        
    vista.vircam.H                                                                                       
    vista.vircam.Ks 
    

### emission lines
    line.NII-654.8 line.NII-654.8_err line.NII-658.4 line.NII-658.4_err 
    line.H-alpha line.H-alpha_err 
    line.OIII-500.7 line.OIII-500.7_err 
    line.H-beta line.H-beta_err

In [25]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from fors2pcigale.fors2starlightio import FILENAME_FORS2PHOTOM,FILENAME_STARLIGHT
from fors2pcigale.fors2starlightio import _getPackageDir
from fors2pcigale.fors2starlightio import FULL_FILENAME_FORS2PHOTOM, FULL_FILENAME_STARLIGHT
from fors2pcigale.fors2starlightio import Fors2DataAcess, SLDataAcess
from fors2pcigale.fors2starlightio import convertflambda_to_fnu, flux_norm,ordered_keys

In [3]:
from astropy import units as u
from astropy import constants as const
import numpy as np

In [4]:
_getPackageDir()

'/Users/dagoret/MacOSX/GitHub/LSST/Fors2pcigale/src/fors2pcigale/fors2starlightio'

In [5]:
FILENAME_FORS2PHOTOM

'data/FORS2spectraGalexKidsPhotom.h5'

In [6]:
FILENAME_STARLIGHT

'data/SLspectra.h5'

In [7]:
FULL_FILENAME_FORS2PHOTOM

'/Users/dagoret/MacOSX/GitHub/LSST/Fors2pcigale/src/fors2pcigale/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5'

In [8]:
!ls /Users/dagoret/MacOSX/GitHub/LSST/Fors2ToStellarPopSynthesis/src/fors2tostellarpopsynthesis/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5

/Users/dagoret/MacOSX/GitHub/LSST/Fors2ToStellarPopSynthesis/src/fors2tostellarpopsynthesis/fors2starlightio/data/FORS2spectraGalexKidsPhotom.h5


In [9]:
J0 = 3631.0 # convert AB flux in Jansky (1 Jansky = 1e-20 erg/cm2/s)

In [10]:
TAG = "SPEC560"

In [11]:
fors2 = Fors2DataAcess()

In [12]:
dir(fors2)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close_file',
 'get_list_of_groupkeys',
 'get_list_subgroup_keys',
 'get_photfluxes',
 'get_photmagnitudes',
 'getattribdata_fromgroup',
 'getspectrum_fromgroup',
 'getspectrumcleanedemissionlines_fromgroup',
 'hf',
 'list_of_groupkeys',
 'list_of_subgroup_keys',
 'plot_allspectra',
 'plot_spectro_photom_noscaling',
 'plot_spectro_photom_rescaling']

In [13]:
list_of_specname = fors2.get_list_of_groupkeys()

In [14]:
for idx,specname in enumerate(list_of_specname):
    print(fors2.get_photfluxes(specname))
    if idx>=1:
        break

(array([6.77413801e-09, 3.48282778e-09,            nan, 7.25752934e-10,
       2.35473479e-09, 4.00341673e-09, 5.63590936e-09, 7.05366452e-09,
       9.07996135e-09, 1.29943452e-08, 1.78360713e-08]), array([1.57494035e-09, 8.18709258e-10,            nan, 4.24668420e-11,
       4.45119259e-11, 1.47622804e-10, 1.83424383e-10, 3.62194927e-10,
       2.93883384e-10, 5.98748019e-10, 6.97714249e-10]))
(array([           nan, 1.35103189e-09, 4.58414065e-10, 6.65999834e-10,
       1.79701182e-09, 3.47277687e-09, 5.01496368e-09, 5.92920575e-09,
       9.30281098e-09, 1.17182542e-08, 1.78949844e-08]), array([           nan, 5.88293786e-10, 1.23928297e-10, 4.13597579e-11,
       4.37090600e-11, 1.45867210e-10, 1.83666168e-10, 3.59159087e-10,
       2.91338699e-10, 5.92897758e-10, 6.88687501e-10]))


In [15]:
for idx,specname in enumerate(list_of_specname):
    print("=========================================================================")
    d = fors2.getattribdata_fromgroup(specname)
    print(d)
    if idx>=1:
        break


OrderedDict([('CLASS_STAR', 0.04272303), ('DECJ2000', -28.447828), ('EXTINCTION_g', 0.05650495), ('EXTINCTION_i', 0.02904796), ('EXTINCTION_r', 0.03908986), ('EXTINCTION_u', 0.07251725), ('FLUXERR_GAAP_H', 594.53876), ('FLUXERR_GAAP_J', 290.61316), ('FLUXERR_GAAP_Ks', 694.3972), ('FLUXERR_GAAP_Y', 356.02774), ('FLUXERR_GAAP_Z', 179.81622), ('FLUXERR_GAAP_g', 3.95e-11), ('FLUXERR_GAAP_i', 1.56e-10), ('FLUXERR_GAAP_r', 4.15e-11), ('FLUXERR_GAAP_u', nan), ('FLUX_GAAP_H', 12903.003), ('FLUX_GAAP_J', 8978.914), ('FLUX_GAAP_Ks', 17751.28), ('FLUX_GAAP_Y', 6933.5605), ('FLUX_GAAP_Z', 5525.0454), ('FLUX_GAAP_g', 6.7505e-10), ('FLUX_GAAP_i', 4.2306e-09), ('FLUX_GAAP_r', 2.1954e-09), ('FLUX_GAAP_u', nan), ('FLUX_RADIUS', 2.967994), ('ID', 'KiDSDR4 J005407.202-282652.18 '), ('KIDS_TILE', 'KIDS_13.5_-28.2 '), ('MAGERR_GAAP_H', 0.0500281), ('MAGERR_GAAP_J', 0.0351411), ('MAGERR_GAAP_Ks', 0.042472), ('MAGERR_GAAP_Y', 0.0557509), ('MAGERR_GAAP_Z', 0.035336), ('MAGERR_GAAP_g', 0.06353097), ('MAGERR_GA

In [16]:
df_f = pd.DataFrame(columns=['redshift','fuv_flux','fuv_fluxerr',
                             'nuv_flux','nuv_fluxerr',
                             'FLUX_GAAP_u','FLUXERR_GAAP_u',
                             'FLUX_GAAP_g','FLUXERR_GAAP_g',
                             'FLUX_GAAP_r','FLUXERR_GAAP_r',
                             'FLUX_GAAP_i','FLUXERR_GAAP_i',
                             'FLUX_GAAP_Z','FLUXERR_GAAP_Z',
                             'FLUX_GAAP_Y','FLUXERR_GAAP_Y',
                             'FLUX_GAAP_J','FLUXERR_GAAP_J',
                             'FLUX_GAAP_H','FLUXERR_GAAP_H',
                             'FLUX_GAAP_Ks','FLUXERR_GAAP_Ks',
                            ], index=list_of_specname)
df_m = pd.DataFrame(columns=['redshift','fuv_mag','fuv_magerr',
                             'nuv_mag','nuv_magerr',
                             'MAG_GAAP_u','MAGERR_GAAP_u',
                             'MAG_GAAP_g','MAGERR_GAAP_g',
                             'MAG_GAAP_r','MAGERR_GAAP_r',
                             'MAG_GAAP_i','MAGERR_GAAP_i',
                             'MAG_GAAP_Z','MAGERR_GAAP_Z',
                             'MAG_GAAP_Y','MAGERR_GAAP_Y',
                             'MAG_GAAP_J','MAGERR_GAAP_J',
                             'MAG_GAAP_H','MAGERR_GAAP_H',
                             'MAG_GAAP_Ks','MAGERR_GAAP_Ks',
                            ], index=list_of_specname)

In [17]:
# apparently flux are in micro-Jansky
for idx,specname in enumerate(list_of_specname):
    d = fors2.getattribdata_fromgroup(specname)
    df_f.loc[specname] = pd.Series({'redshift':d['redshift'], 
                                     'fuv_flux':d['fuv_flux'], 
                                     'fuv_fluxerr':d['fuv_fluxerr'],
                                     'nuv_flux':d['nuv_flux'], 
                                     'nuv_fluxerr':d['nuv_fluxerr'],
                                     'FLUX_GAAP_u':d['FLUX_GAAP_u'],
                                     'FLUXERR_GAAP_u':d['FLUXERR_GAAP_u'],
                                     'FLUX_GAAP_g':d['FLUX_GAAP_g'],
                                     'FLUXERR_GAAP_g':d['FLUXERR_GAAP_g'],
                                     'FLUX_GAAP_r':d['FLUX_GAAP_r'],
                                     'FLUXERR_GAAP_r':d['FLUXERR_GAAP_r'],
                                     'FLUX_GAAP_i':d['FLUX_GAAP_i'],
                                     'FLUXERR_GAAP_i':d['FLUXERR_GAAP_i'],
                                     'FLUX_GAAP_Z':d['FLUX_GAAP_Z'],
                                     'FLUXERR_GAAP_Z':d['FLUXERR_GAAP_Z'],
                                     'FLUX_GAAP_Y':d['FLUX_GAAP_Y'],
                                     'FLUXERR_GAAP_Y':d['FLUXERR_GAAP_Y'],
                                     'FLUX_GAAP_J':d['FLUX_GAAP_J'],
                                     'FLUXERR_GAAP_J':d['FLUXERR_GAAP_J'],
                                     'FLUX_GAAP_H':d['FLUX_GAAP_H'],
                                     'FLUXERR_GAAP_H':d['FLUXERR_GAAP_H'],
                                     'FLUX_GAAP_Ks':d['FLUX_GAAP_Ks'], 
                                     'FLUXERR_GAAP_Ks':d['FLUXERR_GAAP_Ks'], 
                                   })

In [18]:
# magnitudes are AB
for idx,specname in enumerate(list_of_specname):
    d = fors2.getattribdata_fromgroup(specname)
    df_m.loc[specname] = pd.Series({'redshift':d['redshift'], 
                                     'fuv_mag':d['fuv_mag'], 
                                     'fuv_magerr':d['fuv_magerr'],
                                     'nuv_mag':d['nuv_mag'], 
                                     'nuv_magerr':d['nuv_magerr'],
                                     'MAG_GAAP_u':d['MAG_GAAP_u'],
                                     'MAGERR_GAAP_u':d['MAGERR_GAAP_u'],
                                     'MAG_GAAP_g':d['MAG_GAAP_g'],
                                     'MAGERR_GAAP_g':d['MAGERR_GAAP_g'],
                                     'MAG_GAAP_r':d['MAG_GAAP_r'],
                                     'MAGERR_GAAP_r':d['MAGERR_GAAP_r'],
                                     'MAG_GAAP_i':d['MAG_GAAP_i'],
                                     'MAGERR_GAAP_i':d['MAGERR_GAAP_i'],
                                     'MAG_GAAP_Z':d['MAG_GAAP_Z'],
                                     'MAGERR_GAAP_Z':d['MAGERR_GAAP_Z'],
                                     'MAG_GAAP_Y':d['MAG_GAAP_Y'],
                                     'MAGERR_GAAP_Y':d['MAGERR_GAAP_Y'],
                                     'MAG_GAAP_J':d['MAG_GAAP_J'],
                                     'MAGERR_GAAP_J':d['MAGERR_GAAP_J'],
                                     'MAG_GAAP_H':d['MAG_GAAP_H'],
                                     'MAGERR_GAAP_H':d['MAGERR_GAAP_H'],
                                     'MAG_GAAP_Ks':d['MAG_GAAP_Ks'],
                                     'MAGERR_GAAP_Ks':d['MAGERR_GAAP_Ks'],
                                   })

In [19]:
# flux in micro-Jansky
df_f.head()

,redshift,fuv_flux,fuv_fluxerr,nuv_flux,nuv_fluxerr,FLUX_GAAP_u,FLUXERR_GAAP_u,FLUX_GAAP_g,FLUXERR_GAAP_g,FLUX_GAAP_r,...,FLUX_GAAP_Z,FLUXERR_GAAP_Z,FLUX_GAAP_Y,FLUXERR_GAAP_Y,FLUX_GAAP_J,FLUXERR_GAAP_J,FLUX_GAAP_H,FLUXERR_GAAP_H,FLUX_GAAP_Ks,FLUXERR_GAAP_Ks
SPEC2,0.34555,24.59552,5.7169,12.645368,2.971828,NaN,NaN,0.0,0.0,0.0,...,5525.0454,179.81622,6933.5605,356.02774,8978.914,290.61316,12903.003,594.53876,17751.28,694.3972
SPEC3,0.6866,NaN,NaN,4.905293,2.135443,0.0,0.0,0.0,0.0,0.0,...,4912.493164,179.913452,5824.850098,352.837769,9195.825195,287.987885,11632.511719,588.559448,17807.927734,685.337463
SPEC9,0.29266,NaN,NaN,9.029878,2.996898,NaN,NaN,0.0,0.0,0.0,...,3194.795654,185.288483,3145.443359,330.740845,4558.929688,268.486725,5803.071777,579.864075,6177.433594,606.197693
SPEC13,0.6887,NaN,NaN,3.221977,1.612464,0.0,0.0,0.0,0.0,0.0,...,1140.422485,180.381531,442.328766,323.638702,1297.0,260.288177,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,12.55755,3.31995,0.0,0.0,0.0,0.0,0.0,...,3555.818848,123.689148,4437.370117,238.951889,5302.674805,225.177338,6213.995117,544.024658,8170.591309,507.931976


In [20]:
# magnitude AB
df_m

,redshift,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr,MAG_GAAP_u,MAGERR_GAAP_u,MAG_GAAP_g,MAGERR_GAAP_g,MAG_GAAP_r,...,MAG_GAAP_Z,MAGERR_GAAP_Z,MAG_GAAP_Y,MAGERR_GAAP_Y,MAG_GAAP_J,MAGERR_GAAP_J,MAG_GAAP_H,MAGERR_GAAP_H,MAG_GAAP_Ks,MAGERR_GAAP_Ks
SPEC2,0.34555,20.422865,0.252426,21.14517,0.255224,NaN,NaN,22.848028,0.063531,21.570145,...,20.62259,0.035336,20.378963,0.055751,20.10479,0.035141,19.715614,0.050028,19.371752,0.042472
SPEC3,0.6866,NaN,NaN,22.173336,0.472773,23.346855,0.293519,22.941315,0.067426,21.863623,...,20.749331,0.039764,20.567509,0.065768,20.078465,0.034002,19.827843,0.054934,19.368172,0.041784
SPEC9,0.29266,NaN,NaN,21.510794,0.360429,NaN,NaN,23.450844,0.091752,21.977757,...,21.216759,0.062969,21.236706,0.114164,20.840393,0.063942,20.583025,0.108491,20.517725,0.106544
SPEC13,0.6887,NaN,NaN,22.629692,0.543497,23.861921,0.408772,23.616484,0.094022,23.223907,...,22.334183,0.171732,23.365664,0.794401,22.204744,0.217891,NaN,NaN,NaN,NaN
SPEC19,0.48395,NaN,NaN,21.152737,0.287115,24.186991,0.698657,23.808002,0.131907,22.011513,...,21.100159,0.037767,20.862761,0.058467,20.676168,0.046106,20.508581,0.095054,20.214029,0.067496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SPEC734,0.23012,19.537182,0.154001,16.488522,0.022848,22.124575,0.089498,21.183992,0.011145,20.361795,...,19.714937,0.00839,19.57863,0.018246,19.286404,0.011201,19.034557,0.020727,19.066914,0.022884
SPEC735,0.36169,20.512026,0.277843,20.175917,0.297318,23.698568,0.336887,21.267984,0.012186,19.724001,...,18.867546,0.004,18.647512,0.008077,18.40612,0.005235,18.052334,0.008725,17.985865,0.008905
SPEC736,0.55777,19.537182,0.154001,16.488522,0.022848,23.172825,0.247612,23.063156,0.069099,22.264561,...,21.720623,0.058642,21.406158,0.089782,21.366528,0.07179,21.771448,0.240246,21.27824,0.161636
SPEC737,0.52001,21.295267,0.383258,21.704386,0.396922,23.499617,0.333195,22.918249,0.053107,21.776768,...,20.798254,0.024403,20.385408,0.034094,20.17865,0.023225,19.833088,0.03932,19.671566,0.035046


In [21]:
J0*np.power(10.,-0.4*21.152) *1e6

12.566844722373133

In [22]:
0.287115*12.56*0.9210340371976183

3.3214001561303275

In [23]:
np.log(10)/2.5

0.9210340371976183